In [28]:
import requests
import json
import subprocess
import re
import os

from dotenv import load_dotenv
load_dotenv()


True

In [29]:
def generateText(prompt: str) -> str:
    api_url = "https://api.mistral.ai/v1/chat/completions"
    api_key = os.getenv("MISTRAL_API_KEY")
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "mistral-tiny",
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post(api_url, headers=headers, data=json.dumps(data))
    return response.json()["choices"][0]["message"]["content"]

In [30]:
test_prompt = "Donne-moi une blague courte sur developpeurs web."
print(generateText(test_prompt))

Un développeur web est comme un artisan médiéval : ils cherchent des clients, travaillent longtemps pour créer quelque chose, puis leur client ne veut rien payer et leur demande plus de fonctionnalités. Le seul différence, c'est que l'artisan médiéval peut utiliser une fourche pour vaincre ses clients.


In [18]:
def writeFile(path, content):
    with open(path, "w", encoding="utf-8") as file:
        file.write(content)

def launchPythonFile(path):
    result = subprocess.run(["python3", path], capture_output=True, text=True)
    print(result.stdout)
    if result.stderr:
        print("Erreur :", result.stderr)
    return result.stdout, result.stderr

In [ ]:
# llm_response = generateText(function_prompt)
# print("Réponse du LLM :", llm_response)

# # Extract only the first JSON object from the response
# json_match = re.search(r'\{.*\}', llm_response, re.DOTALL)
# if not json_match:
#     raise ValueError("Aucun JSON valide trouvé dans la réponse du LLM.")

# json_str = json_match.group(0)
# reponse_json = json.loads(json_str)
# func_name = reponse_json["function"]
# args = reponse_json["args"]

# if func_name == "writeFile":
#     writeFile(**args)
# elif func_name == "launchPythonFile":
#     launchPythonFile(**args)
# else:
#     print("Fonction non reconnue :", func_name)

In [ ]:
# function_prompt = (
#     "Tu as accès à deux fonctions :\n"
#     "- writeFile(path, content): écrit le contenu dans le fichier spécifié.\n"
#     "- launchPythonFile(path): exécute un fichier Python.\n"
#     "Pour répondre, renvoie un JSON de la forme : "
#     '{"function": "NOM_FONCTION", "args": {"arg1": "valeur", ...}}.\n'
#     "Demande : écris un fichier Python qui affiche 'hello world'."
# )

In [22]:
import json

def extract_json_blocks(text):
    blocks = []
    brace_stack = []
    current_block = ''
    in_block = False

    for char in text:
        if char == '{':
            brace_stack.append(char)
            in_block = True
        if in_block:
            current_block += char
        if char == '}':
            brace_stack.pop()
            if not brace_stack:
                # Fin d’un bloc JSON complet
                blocks.append(current_block)
                current_block = ''
                in_block = False

    return blocks


In [23]:
def run_agent(prompt: str, max_step: int = 5):
    last_result = ""

    for step in range(max_step):
        if last_result:
            prompt_step = prompt + f"\nRésultat de la tâche précédente : {last_result}\n"
        else:
            prompt_step = prompt

        llm_response = generateText(prompt_step)
        print(f"Étape {step+1} - Réponse du LLM :\n{llm_response}")

        json_matches = extract_json_blocks(llm_response)

        if not json_matches:
            print("Aucun JSON valide trouvé dans la réponse du LLM.")
            break

        for json_str in json_matches:
            try:
                reponse_json = json.loads(json_str)
                func_name = reponse_json.get("function")
                args = reponse_json.get("args", {})

                if func_name == "writeFile":
                    writeFile(**args)
                    last_result = f"Fichier {args.get('path')} écrit."
                elif func_name == "launchPythonFile":
                    out, err = launchPythonFile(**args)
                    last_result = f"Sortie : {out}\nErreur : {err}"
                elif func_name == "stop":
                    print("Arrêt demandé par l'agent.")
                    return
                else:
                    print("Fonction non reconnue :", func_name)
                    return
            except json.JSONDecodeError as e:
                print("Erreur de décodage JSON :", e)
                continue


In [24]:
function_prompt = (
    "Tu as accès à trois fonctions :\n"
    "- writeFile(path, content): écrit le contenu dans le fichier spécifié.\n"
    "- launchPythonFile(path): exécute un fichier Python.\n"
    "- stop(): arrête l'agent si la tâche est terminée.\n"
    "Pour chaque étape, renvoie un JSON de la forme : "
    '{"function": "NOM_FONCTION", "args": {"arg1": "valeur", ...}}.\n'
    "Ta tâche : écris un fichier Python qui affiche 'hello world', exécute-le, puis arrête-toi."
)

In [25]:
run_agent(function_prompt, max_step=5)

Étape 1 - Réponse du LLM :
Pour réaliser cette tâche, je vais utiliser les fonctions que tu m'as fournies en trois étapes distinctes. Voici les commandes JSON à exécuter :

1. Écriture du fichier Python :

```
{"function": "writeFile", "args": {"path": "hello_world.py", "content": "print('hello world')"}}
```

2. Exécution du fichier Python :

```
{"function": "launchPythonFile", "args": {"path": "hello_world.py"}}
```

3. Arrêt de l'agent :

```
{"function": "stop"}
```

Ces commandes prennent en charge l'écriture d'un fichier Python contenant la chaîne 'hello world', l'exécution de ce fichier et l'arrêt de l'agent.
hello world

Arrêt demandé par l'agent.
